## 라이브러리

In [1]:
# Colab
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
import copy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import  GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
import numpy as np

## 데이터 로드

In [4]:
train = pd.read_csv('./drive/MyDrive/data/train.csv')
test = pd.read_csv('./drive/MyDrive/data/test.csv')
pd.set_option('display.max_columns', None)

In [30]:
pd.set_option('display.max_rows', None)

In [ ]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
pd.set_option('display.max_columns', None)

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 70 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         15000 non-null  int64  
 1   Q1            14959 non-null  float64
 2   Q2            14931 non-null  float64
 3   Q3            14950 non-null  float64
 4   Q4            14929 non-null  float64
 5   Q5            14962 non-null  float64
 6   Q6            14952 non-null  float64
 7   Q7            14924 non-null  float64
 8   Q8            14952 non-null  float64
 9   Q9            14944 non-null  float64
 10  Q10           14928 non-null  float64
 11  Q11           14941 non-null  float64
 12  Q12           14933 non-null  float64
 13  Q13           14960 non-null  float64
 14  Q14           14964 non-null  float64
 15  Q15           14955 non-null  float64
 16  Q16           14967 non-null  float64
 17  Q17           14963 non-null  float64
 18  Q18           14937 non-nu

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35452 entries, 0 to 35451
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         35452 non-null  int64  
 1   Q1            35333 non-null  float64
 2   Q2            35305 non-null  float64
 3   Q3            35322 non-null  float64
 4   Q4            35327 non-null  float64
 5   Q5            35362 non-null  float64
 6   Q6            35320 non-null  float64
 7   Q7            35310 non-null  float64
 8   Q8            35344 non-null  float64
 9   Q9            35348 non-null  float64
 10  Q10           35232 non-null  float64
 11  Q11           35333 non-null  float64
 12  Q12           35303 non-null  float64
 13  Q13           35356 non-null  float64
 14  Q14           35350 non-null  float64
 15  Q15           35345 non-null  float64
 16  Q16           35367 non-null  float64
 17  Q17           35373 non-null  float64
 18  Q18           35305 non-nu

In [5]:
x_train = train.copy()
# x_train.drop('nerdiness', axis=1, inplace = True)
# y_train = train['nerdiness']

In [6]:
dataset = [x_train, test]

## 이상치 처리

In [7]:
intro_error_big = x_train['introelapse'].quantile(.95)
intro_error_small = x_train['introelapse'].quantile(.003)
survey_error_big = x_train['surveyelapse'].quantile(.95)
survey_error_small = x_train['surveyelapse'].quantile(.003)
test_error_big = x_train['testelapse'].quantile(.95)
test_error_small = x_train['testelapse'].quantile(.003)
print(intro_error_big,intro_error_small,survey_error_big,survey_error_small,test_error_big,test_error_small)

976.0 1.0 381.0 60.0 290.0 42.997


In [8]:
index = x_train[x_train['introelapse'] > intro_error_big].index
x_train['introelapse'].iloc[index] = np.nan
print(index)
index = x_train[x_train['surveyelapse'] > survey_error_big].index
x_train['surveyelapse'].iloc[index] = np.nan
print(index)
index = x_train[x_train['testelapse'] > test_error_big].index
x_train['testelapse'].iloc[index] = np.nan
print(index)
index = x_train[x_train['introelapse'] < intro_error_small].index
x_train['introelapse'].iloc[index] = np.nan
print(index)
index = x_train[x_train['surveyelapse'] < survey_error_small].index
x_train['surveyelapse'].iloc[index] = np.nan
print(index)
index = x_train[x_train['testelapse'] < test_error_small].index
x_train['testelapse'].iloc[index] = np.nan
print(index)

Int64Index([   35,    56,    59,    96,   113,   130,   142,   181,   212,
              227,
            ...
            14796, 14801, 14823, 14862, 14867, 14869, 14892, 14924, 14940,
            14962],
           dtype='int64', length=748)
Int64Index([   43,    48,    60,    74,    89,    90,    92,    97,   112,
              113,
            ...
            14875, 14878, 14887, 14892, 14906, 14939, 14949, 14974, 14984,
            14994],
           dtype='int64', length=744)
Int64Index([    0,     4,    13,    15,    24,    55,    60,    73,    79,
               87,
            ...
            14837, 14839, 14874, 14875, 14878, 14889, 14894, 14898, 14926,
            14991],
           dtype='int64', length=749)
Int64Index([], dtype='int64')
Int64Index([    0,   548,   632,   663,   738,   835,  1775,  1858,  2670,
             2883,  3176,  3712,  3833,  4115,  4766,  4820,  5099,  5467,
             5578,  5692,  6328,  6575,  6811,  7588,  7746,  8799,  9142,
             935

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [9]:
index = test[test['introelapse'] > intro_error_big].index
test['introelapse'].iloc[index] = np.nan
print(index)
index = test[test['surveyelapse'] > survey_error_big].index
test['surveyelapse'].iloc[index] = np.nan
print(index)
index = test[test['testelapse'] > test_error_big].index
test['testelapse'].iloc[index] = np.nan
print(index)
index = test[test['introelapse'] < intro_error_small].index
test['introelapse'].iloc[index] = np.nan
print(index)
index = test[test['surveyelapse'] < survey_error_small].index
test['surveyelapse'].iloc[index] = np.nan
print(index)
index = test[test['testelapse'] < test_error_small].index
test['testelapse'].iloc[index] = np.nan
print(index)

Int64Index([    3,    57,    84,    92,   102,   112,   113,   158,   169,
              170,
            ...
            35218, 35232, 35245, 35246, 35272, 35335, 35368, 35414, 35427,
            35429],
           dtype='int64', length=1890)
Int64Index([   13,    63,    94,   127,   284,   303,   304,   320,   328,
              333,
            ...
            35261, 35286, 35287, 35315, 35365, 35366, 35368, 35385, 35398,
            35425],
           dtype='int64', length=1782)
Int64Index([   33,   151,   164,   204,   244,   263,   266,   278,   280,
              284,
            ...
            35228, 35246, 35269, 35300, 35346, 35351, 35385, 35391, 35435,
            35445],
           dtype='int64', length=1821)
Int64Index([21768, 33952], dtype='int64')
Int64Index([  569,   657,   681,  2307,  2504,  3137,  3372,  4337,  4383,
             4969,  5351,  5805,  6029,  6218,  6297,  6413,  6441,  6937,
             8133,  9442, 10077, 10098, 10284, 10364, 10462, 10782, 11341,
 

In [10]:
index = x_train[x_train['familysize'] > 23].index
x_train['familysize'].iloc[index] = np.nan
print(index)

index = test[test['familysize'] > 23].index
test['familysize'].iloc[index] = np.nan
print(index)

Int64Index([1019, 1287, 9817], dtype='int64')
Int64Index([4665, 5125, 6475, 10075, 11773, 25389, 30771], dtype='int64')


In [11]:
index = x_train[x_train['age'] > 100].index
x_train['age'].iloc[index] = np.nan
print(index)

index = test[test['age'] > 100].index
test['age'].iloc[index] = np.nan
print(index)

Int64Index([1739, 4002, 4976, 6725, 9273, 14135], dtype='int64')
Int64Index([ 2578,  2802,  4609,  8190,  8625, 11922, 13943, 17311, 24301,
            26070, 26332, 28448, 30699, 33783, 33787, 34046],
           dtype='int64')


In [12]:
x_train.head(20)

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,country,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,1.0,5.0,1.0,5.0,1.0,1.0,USA,3.0,NaN,NaN,4.0,3.0,5.0,1.0,3.0,5.0,5.0,3.0,5.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,2.0,1,3.0,1.0,20.0,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,1.0,4.0,5.0,3.0,1.0,2.0,4.0,5.0,1.0,3.0,1.0,1.0,5.0,3.0,2.0,5.0,USA,5.0,85.0,120.0,4.0,2.0,3.0,5.0,3.0,2.0,5.0,1.0,2.0,2.0,1,1,1,1,1,0,1,0,0,1,0,0,1,1,1,1,4.0,2,2.0,1.0,49.0,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,2.0,5.0,5.0,5.0,1.0,3.0,5.0,3.0,5.0,2.0,2.0,1.0,2.0,4.0,2.0,5.0,NLD,9.0,108.0,100.0,1.0,2.0,3.0,1.0,5.0,5.0,3.0,4.0,5.0,2.0,1,1,0,1,1,0,1,1,0,1,0,0,1,1,1,1,2.0,1,1.0,2.0,43.0,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,5.0,2.0,2.0,4.0,4.0,2.0,4.0,5.0,4.0,3.0,3.0,4.0,3.0,4.0,4.0,2.0,USA,2.0,121.0,139.0,3.0,3.0,3.0,4.0,5.0,3.0,4.0,4.0,3.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,1.0,3,1.0,1.0,17.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,4.0,4.0,3.0,5.0,5.0,2.0,4.0,1.0,4.0,2.0,4.0,2.0,3.0,4.0,4.0,4.0,ITA,3.0,NaN,216.0,3.0,3.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,2.0,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,1,1.0,2,2.0,2.0,18.0,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
5,5,5.0,4.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,3.0,4.0,5.0,2.0,3.0,4.0,1.0,1.0,2.0,4.0,2.0,2.0,2.0,3.0,5.0,3.0,5.0,USA,3.0,100.0,176.0,5.0,3.0,3.0,3.0,5.0,2.0,4.0,5.0,3.0,1.0,1,1,0,1,1,0,0,1,0,1,0,0,1,1,1,1,3.0,2,1.0,1.0,26.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
6,6,4.0,3.0,4.0,3.0,5.0,4.0,5.0,4.0,5.0,5.0,3.0,5.0,5.0,3.0,3.0,3.0,5.0,5.0,3.0,4.0,4.0,3.0,5.0,5.0,3.0,5.0,USA,17.0,88.0,164.0,3.0,2.0,5.0,3.0,4.0,3.0,3.0,2.0,3.0,2.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,4.0,3,2.0,2.0,40.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1
7,7,4.0,5.0,4.0,4.0,4.0,4.0,2.0,5.0,3.0,4.0,2.0,2.0,3.0,3.0,5.0,2.0,4.0,5.0,2.0,3.0,5.0,3.0,5.0,5.0,1.0,4.0,NLD,20.0,53.0,112.0,3.0,2.0,5.0,5.0,3.0,3.0,3.0,2.0,2.0,2.0,1,1,1,1,0,0,0,1,0,1,1,0,1,1,1,1,3.0,1,2.0,2.0,34.0,1.0,2.0,5.0,1.0,1.0,2.0,2.0,1
8,8,4.0,4.0,3.0,4.0,4.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,3.0,3.0,1.0,2.0,4.0,1.0,3.0,1.0,2.0,5.0,3.0,1.0,4.0,USA,9.0,164.0,213.0,3.0,1.0,4.0,3.0,4.0,3.0,4.0,1.0,3.0,3.0,1,1,1,1,1,1,0,0,0,1,0,0,1,1,1,1,2.0,2,2.0,1.0,20.0,1.0,7.0,1.0,1.0,1.0,3.0,2.0,0
9,9,3.0,3.0,4.0,3.0,4.0,2.0,4.0,2.0,4.0,4.0,5.0,4.0,1.0,4.0,2.0,5.0,4.0,4.0,5.0,3.0,1.0,1.0,2.0,4.0,4.0,2.0,ARE,109.0,134.0,177.0,2.0,2.0,5.0,3.0,3.0,4.0,4.0,2.0,4.0,4.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,2.0,2,1.0,2.0,17.0,1.0,10.0,1.0,2.0,1.0,5.0,2.0,0


In [13]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 70 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         15000 non-null  int64  
 1   Q1            14959 non-null  float64
 2   Q2            14931 non-null  float64
 3   Q3            14950 non-null  float64
 4   Q4            14929 non-null  float64
 5   Q5            14962 non-null  float64
 6   Q6            14952 non-null  float64
 7   Q7            14924 non-null  float64
 8   Q8            14952 non-null  float64
 9   Q9            14944 non-null  float64
 10  Q10           14928 non-null  float64
 11  Q11           14941 non-null  float64
 12  Q12           14933 non-null  float64
 13  Q13           14960 non-null  float64
 14  Q14           14964 non-null  float64
 15  Q15           14955 non-null  float64
 16  Q16           14967 non-null  float64
 17  Q17           14963 non-null  float64
 18  Q18           14937 non-nu

In [14]:
for data in dataset:
    data.drop('index', axis=1, inplace = True)

## 결측치 처리

In [15]:
#country피처의 최빈값으로 결측치 채운다
for data in dataset:
    data['country'] = data['country'].fillna(x_train['country'].mode()[0])

In [16]:
#country피처 원핫 인코딩
# country_list = ['USA','GBR','CAN','AUS','DEU','PHL','BRA','POL','IND','FRA','ITA','NLD']
# enc = OneHotEncoder(categories=[country_list],handle_unknown='ignore')

# for data in dataset:
#     temp = enc.fit_transform(data['country'].values.reshape(-1,1)).toarray()
#     temp = pd.DataFrame(temp,columns=country_list)
#     data[country_list] = temp.copy()
#     data.drop('country', axis=1, inplace = True)

In [17]:
country_list = ['USA','GBR','CAN','AUS','DEU','PHL','BRA','POL','IND','FRA','ITA','NLD']
enc = OneHotEncoder(categories=[country_list],handle_unknown='ignore')

temp = enc.fit_transform(x_train['country'].values.reshape(-1,1)).toarray()
temp = pd.DataFrame(temp,columns=country_list)
x_train[country_list] = temp.copy()

temp = enc.transform(test['country'].values.reshape(-1,1)).toarray()
temp = pd.DataFrame(temp,columns=country_list)
test[country_list] = temp.copy()

x_train.drop('country', axis=1, inplace = True)  
test.drop('country', axis=1, inplace = True)  

In [18]:
#knn을 이용하여 나머지 결측치 채우기
# imputer = KNNImputer(n_neighbors=5)
# column_list1 = x_train.columns.tolist()
# column_list2 = test.columns.tolist()
# x_train[column_list1] = imputer.fit_transform(x_train[column_list1])
# test[column_list2] = imputer.fit_transform(test[column_list2])

In [19]:
imputer = KNNImputer(n_neighbors=5)
column_list1 = x_train.columns.tolist()
column_list2 = column_list1.copy()
column_list2.remove('nerdiness')
print(column_list1)
print(column_list2)
x_train[column_list1] = imputer.fit_transform(x_train[column_list1])
imputer.fit(x_train[column_list2])
test[column_list2] = imputer.transform(test[column_list2])

['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'introelapse', 'testelapse', 'surveyelapse', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4', 'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1', 'VCL2', 'VCL3', 'VCL4', 'VCL5', 'VCL6', 'VCL7', 'VCL8', 'VCL9', 'VCL10', 'VCL11', 'VCL12', 'VCL13', 'VCL14', 'VCL15', 'VCL16', 'education', 'urban', 'gender', 'engnat', 'age', 'hand', 'religion', 'orientation', 'voted', 'married', 'familysize', 'ASD', 'nerdiness', 'USA', 'GBR', 'CAN', 'AUS', 'DEU', 'PHL', 'BRA', 'POL', 'IND', 'FRA', 'ITA', 'NLD']
['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'introelapse', 'testelapse', 'surveyelapse', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4', 'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1', 'VCL2', 'VCL3',

In [20]:
x_train.head(20)

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness,USA,GBR,CAN,AUS,DEU,PHL,BRA,POL,IND,FRA,ITA,NLD
0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,1.0,5.0,1.0,5.0,1.0,1.0,3.0,97.4,163.4,4.0,3.0,5.0,1.0,3.0,5.0,5.0,3.0,5.0,3.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,20.0,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,1.0,4.0,5.0,3.0,1.0,2.0,4.0,5.0,1.0,3.0,1.0,1.0,5.0,3.0,2.0,5.0,5.0,85.0,120.0,4.0,2.0,3.0,5.0,3.0,2.0,5.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,4.0,2.0,2.0,1.0,49.0,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,2.0,5.0,5.0,5.0,1.0,3.0,5.0,3.0,5.0,2.0,2.0,1.0,2.0,4.0,2.0,5.0,9.0,108.0,100.0,1.0,2.0,3.0,1.0,5.0,5.0,3.0,4.0,5.0,2.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,43.0,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,5.0,2.0,2.0,4.0,4.0,2.0,4.0,5.0,4.0,3.0,3.0,4.0,3.0,4.0,4.0,2.0,2.0,121.0,139.0,3.0,3.0,3.0,4.0,5.0,3.0,4.0,4.0,3.0,3.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,17.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,4.0,4.0,3.0,5.0,5.0,2.0,4.0,1.0,4.0,2.0,4.0,2.0,3.0,4.0,4.0,4.0,3.0,131.0,216.0,3.0,3.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,2.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,18.0,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,5.0,4.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,3.0,4.0,5.0,2.0,3.0,4.0,1.0,1.0,2.0,4.0,2.0,2.0,2.0,3.0,5.0,3.0,5.0,3.0,100.0,176.0,5.0,3.0,3.0,3.0,5.0,2.0,4.0,5.0,3.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,26.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4.0,3.0,4.0,3.0,5.0,4.0,5.0,4.0,5.0,5.0,3.0,5.0,5.0,3.0,3.0,3.0,5.0,5.0,3.0,4.0,4.0,3.0,5.0,5.0,3.0,5.0,17.0,88.0,164.0,3.0,2.0,5.0,3.0,4.0,3.0,3.0,2.0,3.0,2.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,4.0,3.0,2.0,2.0,40.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4.0,5.0,4.0,4.0,4.0,4.0,2.0,5.0,3.0,4.0,2.0,2.0,3.0,3.0,5.0,2.0,4.0,5.0,2.0,3.0,5.0,3.0,5.0,5.0,1.0,4.0,20.0,53.0,112.0,3.0,2.0,5.0,5.0,3.0,3.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,2.0,34.0,1.0,2.0,5.0,1.0,1.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,4.0,4.0,3.0,4.0,4.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,3.0,3.0,1.0,2.0,4.0,1.0,3.0,1.0,2.0,5.0,3.0,1.0,4.0,9.0,164.0,213.0,3.0,1.0,4.0,3.0,4.0,3.0,4.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,20.0,1.0,7.0,1.0,1.0,1.0,3.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,3.0,3.0,4.0,3.0,4.0,2.0,4.0,2.0,4.0,4.0,5.0,4.0,1.0,4.0,2.0,5.0,4.0,4.0,5.0,3.0,1.0,1.0,2.0,4.0,4.0,2.0,109.0,134.0,177.0,2.0,2.0,5.0,3.0,3.0,4.0,4.0,2.0,4.0,4.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,17.0,1.0,10.0,1.0,2.0,1.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 80 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            15000 non-null  float64
 1   Q2            15000 non-null  float64
 2   Q3            15000 non-null  float64
 3   Q4            15000 non-null  float64
 4   Q5            15000 non-null  float64
 5   Q6            15000 non-null  float64
 6   Q7            15000 non-null  float64
 7   Q8            15000 non-null  float64
 8   Q9            15000 non-null  float64
 9   Q10           15000 non-null  float64
 10  Q11           15000 non-null  float64
 11  Q12           15000 non-null  float64
 12  Q13           15000 non-null  float64
 13  Q14           15000 non-null  float64
 14  Q15           15000 non-null  float64
 15  Q16           15000 non-null  float64
 16  Q17           15000 non-null  float64
 17  Q18           15000 non-null  float64
 18  Q19           15000 non-nu

In [22]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35452 entries, 0 to 35451
Data columns (total 79 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            35452 non-null  float64
 1   Q2            35452 non-null  float64
 2   Q3            35452 non-null  float64
 3   Q4            35452 non-null  float64
 4   Q5            35452 non-null  float64
 5   Q6            35452 non-null  float64
 6   Q7            35452 non-null  float64
 7   Q8            35452 non-null  float64
 8   Q9            35452 non-null  float64
 9   Q10           35452 non-null  float64
 10  Q11           35452 non-null  float64
 11  Q12           35452 non-null  float64
 12  Q13           35452 non-null  float64
 13  Q14           35452 non-null  float64
 14  Q15           35452 non-null  float64
 15  Q16           35452 non-null  float64
 16  Q17           35452 non-null  float64
 17  Q18           35452 non-null  float64
 18  Q19           35452 non-nu

## 피처엔지니어링

In [23]:
Answers1 = ['Q1','Q2','Q3','Q4','Q5',
             'Q6','Q7','Q8','Q9','Q10', 
             'Q11','Q12','Q13','Q14','Q15', 
             'Q16','Q17','Q18','Q19','Q20',
             'Q21','Q22','Q23','Q24','Q25',
             'Q26']

Answers1_1 = ['Q1','Q2','Q3','Q4','Q5',
             'Q6','Q7','Q8','Q9','Q10', 
             'Q11','Q12','Q13','Q14','Q15', 
             'Q16','Q17','Q18','Q19','Q20']

Answers1_2 = ['Q21','Q22','Q23','Q24','Q25',
             'Q26']

Answers2 = ['TIPI1', 'TIPI2', 'TIPI3', 'TIPI4', 'TIPI5',
            'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10']
            
Answers3 = ['VCL1','VCL2','VCL3','VCL4','VCL5',
            'VCL6','VCL7','VCL8','VCL9','VCL10',
            'VCL11','VCL12','VCL13','VCL14','VCL15',
            'VCL16']

In [24]:
#Q로 T/V/M 나누기
for data in dataset:
  data['T'] = data['Q1'] + data['Q2'] + data['Q3'] + data['Q15']
  data['V'] = data['Q4'] + data['Q5'] + data['Q7'] + data['Q8'] + data['Q10'] + data['Q11'] + data['Q12']  + data['Q13'] + data['Q17'] + data['Q19'] + data['Q20'] 
  data['M'] = data['Q9'] + data['Q16']

In [25]:
#Q질문 이용하여 점수 계산(Q26개 설문의 답)
Ancoms = list(combinations(Answers1, 2))
for data in dataset:
    data['Q_score'] = data[Answers1].mean(axis = 1)
    data['Q_var'] = data[Answers1].var(axis=1)
    data['Q_score_1'] = data[Answers1_1].mean(axis = 1)
    data['Q_var_1'] = data[Answers1_1].var(axis=1)
    data['Q_score_2'] = data[Answers1_2].mean(axis = 1)
    data['Q_var_2'] = data[Answers1_2].var(axis=1)
    temp_dict = {}
    for a,b in Ancoms:
        temp_dict['%s_dv_%s'%(a,b)] = data[a]/data[b]
    data[list(temp_dict.keys())] = pd.DataFrame(temp_dict).copy()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]


In [26]:
dataset[0].describe()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness,USA,GBR,CAN,AUS,DEU,PHL,BRA,POL,IND,FRA,ITA,NLD,T,V,M,Q_score,Q_var,Q_score_1,Q_var_1,Q_score_2,Q_var_2,Q1_dv_Q2,Q1_dv_Q3,Q1_dv_Q4,Q1_dv_Q5,Q1_dv_Q6,Q1_dv_Q7,Q1_dv_Q8,Q1_dv_Q9,Q1_dv_Q10,Q1_dv_Q11,Q1_dv_Q12,Q1_dv_Q13,Q1_dv_Q14,Q1_dv_Q15,Q1_dv_Q16,Q1_dv_Q17,Q1_dv_Q18,Q1_dv_Q19,Q1_dv_Q20,Q1_dv_Q21,Q1_dv_Q22,Q1_dv_Q23,Q1_dv_Q24,Q1_dv_Q25,Q1_dv_Q26,Q2_dv_Q3,Q2_dv_Q4,Q2_dv_Q5,Q2_dv_Q6,Q2_dv_Q7,Q2_dv_Q8,Q2_dv_Q9,Q2_dv_Q10,Q2_dv_Q11,Q2_dv_Q12,Q2_dv_Q13,Q2_dv_Q14,Q2_dv_Q15,Q2_dv_Q16,Q2_dv_Q17,Q2_dv_Q18,Q2_dv_Q19,Q2_dv_Q20,Q2_dv_Q21,Q2_dv_Q22,Q2_dv_Q23,Q2_dv_Q24,Q2_dv_Q25,Q2_dv_Q26,Q3_dv_Q4,Q3_dv_Q5,Q3_dv_Q6,Q3_dv_Q7,Q3_dv_Q8,Q3_dv_Q9,Q3_dv_Q10,Q3_dv_Q11,Q3_dv_Q12,Q3_dv_Q13,Q3_dv_Q14,Q3_dv_Q15,Q3_dv_Q16,Q3_dv_Q17,Q3_dv_Q18,Q3_dv_Q19,Q3_dv_Q20,Q3_dv_Q21,Q3_dv_Q22,Q3_dv_Q23,Q3_dv_Q24,Q3_dv_Q25,Q3_dv_Q26,Q4_dv_Q5,Q4_dv_Q6,Q4_dv_Q7,Q4_dv_Q8,Q4_dv_Q9,Q4_dv_Q10,Q4_dv_Q11,Q4_dv_Q12,Q4_dv_Q13,Q4_dv_Q14,Q4_dv_Q15,Q4_dv_Q16,Q4_dv_Q17,Q4_dv_Q18,Q4_dv_Q19,Q4_dv_Q20,Q4_dv_Q21,Q4_dv_Q22,Q4_dv_Q23,Q4_dv_Q24,Q4_dv_Q25,Q4_dv_Q26,Q5_dv_Q6,Q5_dv_Q7,Q5_dv_Q8,Q5_dv_Q9,Q5_dv_Q10,Q5_dv_Q11,Q5_dv_Q12,Q5_dv_Q13,Q5_dv_Q14,Q5_dv_Q15,Q5_dv_Q16,Q5_dv_Q17,Q5_dv_Q18,Q5_dv_Q19,Q5_dv_Q20,Q5_dv_Q21,Q5_dv_Q22,Q5_dv_Q23,Q5_dv_Q24,Q5_dv_Q25,Q5_dv_Q26,Q6_dv_Q7,Q6_dv_Q8,Q6_dv_Q9,Q6_dv_Q10,Q6_dv_Q11,Q6_dv_Q12,Q6_dv_Q13,Q6_dv_Q14,Q6_dv_Q15,Q6_dv_Q16,Q6_dv_Q17,Q6_dv_Q18,Q6_dv_Q19,Q6_dv_Q20,Q6_dv_Q21,Q6_dv_Q22,Q6_dv_Q23,Q6_dv_Q24,Q6_dv_Q25,Q6_dv_Q26,Q7_dv_Q8,Q7_dv_Q9,Q7_dv_Q10,Q7_dv_Q11,Q7_dv_Q12,Q7_dv_Q13,Q7_dv_Q14,Q7_dv_Q15,Q7_dv_Q16,Q7_dv_Q17,Q7_dv_Q18,Q7_dv_Q19,Q7_dv_Q20,Q7_dv_Q21,Q7_dv_Q22,Q7_dv_Q23,Q7_dv_Q24,Q7_dv_Q25,Q7_dv_Q26,Q8_dv_Q9,Q8_dv_Q10,Q8_dv_Q11,Q8_dv_Q12,Q8_dv_Q13,Q8_dv_Q14,Q8_dv_Q15,Q8_dv_Q16,Q8_dv_Q17,Q8_dv_Q18,Q8_dv_Q19,Q8_dv_Q20,Q8_dv_Q21,Q8_dv_Q22,Q8_dv_Q23,Q8_dv_Q24,Q8_dv_Q25,Q8_dv_Q26,Q9_dv_Q10,Q9_dv_Q11,Q9_dv_Q12,Q9_dv_Q13,Q9_dv_Q14,Q9_dv_Q15,Q9_dv_Q16,Q9_dv_Q17,Q9_dv_Q18,Q9_dv_Q19,Q9_dv_Q20,Q9_dv_Q21,Q9_dv_Q22,Q9_dv_Q23,Q9_dv_Q24,Q9_dv_Q25,Q9_dv_Q26,Q10_dv_Q11,Q10_dv_Q12,Q10_dv_Q13,Q10_dv_Q14,Q10_dv_Q15,Q10_dv_Q16,Q10_dv_Q17,Q10_dv_Q18,Q10_dv_Q19,Q10_dv_Q20,Q10_dv_Q21,Q10_dv_Q22,Q10_dv_Q23,Q10_dv_Q24,Q10_dv_Q25,Q10_dv_Q26,Q11_dv_Q12,Q11_dv_Q13,Q11_dv_Q14,Q11_dv_Q15,Q11_dv_Q16,Q11_dv_Q17,Q11_dv_Q18,Q11_dv_Q19,Q11_dv_Q20,Q11_dv_Q21,Q11_dv_Q22,Q11_dv_Q23,Q11_dv_Q24,Q11_dv_Q25,Q11_dv_Q26,Q12_dv_Q13,Q12_dv_Q14,Q12_dv_Q15,Q12_dv_Q16,Q12_dv_Q17,Q12_dv_Q18,Q12_dv_Q19,Q12_dv_Q20,Q12_dv_Q21,Q12_dv_Q22,Q12_dv_Q23,Q12_dv_Q24,Q12_dv_Q25,Q12_dv_Q26,Q13_dv_Q14,Q13_dv_Q15,Q13_dv_Q16,Q13_dv_Q17,Q13_dv_Q18,Q13_dv_Q19,Q13_dv_Q20,Q13_dv_Q21,Q13_dv_Q22,Q13_dv_Q23,Q13_dv_Q24,Q13_dv_Q25,Q13_dv_Q26,Q14_dv_Q15,Q14_dv_Q16,Q14_dv_Q17,Q14_dv_Q18,Q14_dv_Q19,Q14_dv_Q20,Q14_dv_Q21,Q14_dv_Q22,Q14_dv_Q23,Q14_dv_Q24,Q14_dv_Q25,Q14_dv_Q26,Q15_dv_Q16,Q15_dv_Q17,Q15_dv_Q18,Q15_dv_Q19,Q15_dv_Q20,Q15_dv_Q21,Q15_dv_Q22,Q15_dv_Q23,Q15_dv_Q24,Q15_dv_Q25,Q15_dv_Q26,Q16_dv_Q17,Q16_dv_Q18,Q16_dv_Q19,Q16_dv_Q20,Q16_dv_Q21,Q16_dv_Q22,Q16_dv_Q23,Q16_dv_Q24,Q16_dv_Q25,Q16_dv_Q26,Q17_dv_Q18,Q17_dv_Q19,Q17_dv_Q20,Q17_dv_Q21,Q17_dv_Q22,Q17_dv_Q23,Q17_dv_Q24,Q17_dv_Q25,Q17_dv_Q26,Q18_dv_Q19,Q18_dv_Q20,Q18_dv_Q21,Q18_dv_Q22,Q18_dv_Q23,Q18_dv_Q24,Q18_dv_Q25,Q18_dv_Q26,Q19_dv_Q20,Q19_dv_Q21,Q19_dv_Q22,Q19_dv_Q23,Q19_dv_Q24,Q19_dv_Q25,Q19_dv_Q26,Q20_dv_Q21,Q20_dv_Q22,Q20_dv_Q23,Q20_dv_Q24,Q20_dv_Q25,Q20_dv_Q26,Q21_dv_Q22,Q21_dv_Q23,Q21_dv_Q24,Q21_dv_Q25,Q21_dv_Q26,Q22_dv_Q23,Q22_dv_Q24,Q22_dv_Q25,Q22_dv_Q26,Q23_dv_Q24,Q23_dv_Q25,Q23_dv_Q26,Q24_dv_Q25,Q24_dv_Q26,Q25_dv_Q26
count,15000.000000,15000.000000,15000.000000,15000.00000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.

In [27]:
#해당 column의 값을 뒤집는다 1~5 -> 5~1
flipping_columns = ["TIPI6", "TIPI2", "TIPI8", "TIPI4", "TIPI10"]

for data in dataset:
    for flip in flipping_columns:
        data[flip] = 6 - data[flip]

In [28]:
# TIPI test에 따라 항목 나누기  외향적 1~5   내향적 1~5 -> 5~1   내향적이 낮다 -> 외향적이 높다로 변환하였음 총 점수 2~10 가능 이를 /2 하여 1~5 값으로 통일
# 외향성, 친화성, 양심성, 정서적 안정, 개방성
for data in dataset:
    data['Ex'] = (data['TIPI1']+data['TIPI6'])/2
    data['Ag'] = (data['TIPI7']+data['TIPI2'])/2
    data['Con'] = (data['TIPI3']+data['TIPI8'])/2
    data['Es'] = (data['TIPI9']+data['TIPI4'])/2
    data['Op'] = (data['TIPI5']+data['TIPI10'])/2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1

In [29]:
for data in dataset:
    temp = data[data['gender'] == 3.0].copy()
    temp['gender'] = 1.5
    data[data['gender'] == 3.0] = temp.copy()

In [30]:
#gender 3값 처리 필요 -> 1.5로 데이터 변환하였음
for data in dataset:
    data['Es_gender'] = data['Es']*data['gender']
    data['Con_gender'] = data['Con']*data['gender']
    data['Op_gender'] = data['Op']*data['gender']

In [31]:
#VLC질문을 이용하여 점수계산(단어의 정의 알고있는지 유무)
#VLC_score sum할 때 어려운 단어에 가중치 주기?
#VLC질문으로 스코어 냈으니 기존 VLC1~VLC16이 필요할까?
for data in dataset:
    data['VLC_score'] = data[Answers3].sum(axis=1)       
    data['VLC_var'] = data[Answers3].var(axis=1)
    data['VLC_mean'] = data[Answers3].mean(axis = 1)   

In [32]:
#지연시간
#기존의 elapse 데이터 필요할까? OR 각각의 데이터를 분포하게 만들기?
for data in dataset:
    data['delay'] = data[['introelapse','testelapse','surveyelapse']].sum(axis=1)
    data['delay'] = data['delay'] ** (1/10)
    data['delay_var'] = data['delay'].var()
    data['introelapse_delay_var'] = data['introelapse'].var()
    data['testelapse_delay_var'] = data['testelapse'].var()
    data['surveyelapse_delay_var'] = data['surveyelapse'].var()
    data['introelapse'] = data['introelapse'] ** (1/10)
    data['testelapse'] = data['introelapse'] ** (1/10)
    data['surveyelapse'] = data['introelapse'] ** (1/10)
    # data.drop('introelapse', axis=1, inplace = True)
    # data.drop('testelapse', axis=1, inplace = True)
    # data.drop('surveyelapse', axis=1, inplace = True)

In [33]:
# 13~18,19~29,30이상 피처 생성
def age_grouping1(age):
    if (age <= 18 ):
        age = '18low'
    elif (age > 18 and age<=29):
        age = '18upper29low'
    else:
        age = '30upper'
    return age
for data in dataset:
  data['age_group1'] = data['age'].apply(age_grouping1)

In [34]:
#age로 age_group object 생성
def age_grouping2(age):
    age = (age//10.)*10
    if(age >= 70):
        age = '+70s'
    else:
        age = str(int(age))+'s'
    return age
for data in dataset:
  data['age_group2'] = data['age'].apply(age_grouping2)

In [35]:
#라벨링 ex) 라벨 인코더
# for data in dataset:
#   encoder = LabelEncoder()
#   data['age_group1'] = encoder.fit_transform(data['age_group1'])
#   data['age_group2'] = encoder.fit_transform(data['age_group2'])

In [36]:
encoder = LabelEncoder()
x_train['age_group1'] = encoder.fit_transform(x_train['age_group1'])
test['age_group1'] = encoder.transform(test['age_group1'])
x_train['age_group2'] = encoder.fit_transform(x_train['age_group2'])
test['age_group2'] = encoder.transform(test['age_group2'])

In [37]:
for data in dataset:
    data.drop('age', axis=1, inplace = True)
    #data.drop('index', axis=1, inplace = True)

In [38]:
y_train = x_train['nerdiness']
x_train.drop('nerdiness', axis=1, inplace = True)

In [37]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Columns: 430 entries, Q1 to age_group2
dtypes: float64(428), int64(2)
memory usage: 49.2 MB


In [38]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35452 entries, 0 to 35451
Columns: 430 entries, Q1 to age_group2
dtypes: float64(428), int64(2)
memory usage: 116.3 MB


## PYCARET 모델 찾기

In [ ]:
!pip install pycaret

In [ ]:
!pip install markupsafe==2.0.1

In [ ]:
from pycaret.classification import *

In [ ]:
temp = pd.concat([x_train,y_train],axis=1)

In [ ]:
clf = setup(data = temp, target = 'nerdiness', session_id=123,feature_selection = True, ignore_low_variance =True, use_gpu = True)

,Description,Value
0,session_id,123
1,Target,nerdiness
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 427)"
5,Missing Values,False
6,Numeric Features,396
7,Categorical Features,30
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='nerdiness',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_st...
                ('fix_perfect', Remove_100(target='nerdiness')),
                ('clean_names', Clean_Colum_Names()),
                ('feature_select',
                 Adv

In [ ]:
best = compare_models(sort = "AUC")

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.7694,0.8579,0.8098,0.7819,0.7955,0.5313,0.5320,2.709
rf,Random Forest Classifier,0.7659,0.8518,0.8053,0.7797,0.7921,0.5243,0.5251,3.182
lightgbm,Light Gradient Boosting Machine,0.7543,0.8266,0.8143,0.7597,0.7859,0.4983,0.5002,1.755
gbc,Gradient Boosting Classifier,0.7378,0.8098,0.8048,0.7434,0.7728,0.4640,0.4662,26.189
lda,Linear Discriminant Analysis,0.7216,0.7928,0.8019,0.7248,0.7613,0.4293,0.4326,1.013
ada,Ada Boost Classifier,0.7223,0.7911,0.7967,0.7281,0.7607,0.4314,0.4342,4.972
lr,Logistic Regression,0.7183,0.7910,0.7871,0.7269,0.7558,0.4241,0.4261,3.814
knn,K Neighbors Classifier,0.6873,0.7352,0.7742,0.6957,0.7328,0.3585,0.3617,6.480
dt,Decision Tree Classifier,0.6974,0.6934,0.7305,0.7253,0.7278,0.3872,0.3874,1.490
nb,Naive Bayes,0.6403,0.6737,0.7613,0.6499,0.7010,0.2568,0.2621,0.075


INFO:logs:create_model_container: 14
INFO:logs:master_model_container: 14
INFO:logs:display_container: 2
INFO:logs:ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=123, verbose=0,
                     warm_start=False)
INFO:logs:compare_models() succesfully completed......................................


## 피처 중요도 확인

In [73]:
model = ExtraTreesClassifier()

model.fit(x_train_test, y_train_test)
#model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=100, verbose=False)
val_pred = model.predict_proba(x_val_test)
val_pred = val_pred[:,1]
score = roc_auc_score(y_val_test, val_pred)
feat_imp = pd.Series(model.feature_importances_, index=dataset[0].columns.tolist()).sort_values(ascending=False)

In [74]:
score

0.8600809178371622

In [75]:
print(feat_imp)

Q_score_1                 0.018377
T                         0.016641
Q_score                   0.016525
V                         0.015331
Q6                        0.013656
Q1                        0.009281
Q15                       0.008686
Q_score_2                 0.008207
Q12                       0.008018
Q4                        0.007584
Q26                       0.007013
Q2                        0.007005
Q24                       0.005800
Q13                       0.005050
Q14                       0.004993
Q8                        0.004896
Q11                       0.004858
Q3                        0.004692
Q20                       0.004636
Q9                        0.004590
Q5                        0.004542
M                         0.004517
Q23                       0.004381
Q7                        0.004283
Q_var_1                   0.003637
Q18                       0.003465
Q3_dv_Q6                  0.003195
Q19                       0.003093
Q3_dv_Q15           

## Extra Trees 파라미터 최적화

In [ ]:
params = {"n_estimators": [60,80,100,120,140,160,180,200],
             "criterion" : ['gini', 'entropy', 'log_loss'],
             "max_depth" : [None,10,20,30,40,50,60,70,80,90,100,110],
             "min_samples_split" :[1,2,3],
             "n_jobs" : [-1]
              }   

In [ ]:
model = ExtraTreesClassifier()
x_train_test, x_val_test, y_train_test, y_val_test = train_test_split(x_train, y_train, random_state=4040)

grid_tree  = GridSearchCV(model, param_grid=params, scoring='accuracy',cv=3, refit=True)
grid_tree .fit( x_train_test, y_train_test )

print('best parameters : ', grid_tree.best_params_)
print('best score : ', grid_tree.best_score_)
em = grid_tree.best_estimator_
pred = em.predict(x_val_test)
accuracy_score(y_val_test, pred)

## LGBM 4개 모델

In [ ]:
def lgbm_rfe_4040(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=4040)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [ ]:
lgbm_archive_4040 = lgbm_rfe_4040(x_train, y_train)

426 0.8375748160087314
383 0.8382326119859476
344 0.8370342880835202
309 0.8382481708921101
278 0.8375511895215954
250 0.8354017554479942
225 0.8332114072137428
202 0.8382539334499483
181 0.8353101307783689
162 0.8350320873626819
145 0.8335199921859715
130 0.8305591899687699
117 0.8334517058755904
105 0.8332684565363396
94 0.8293274432308817
84 0.8297452286741421
75 0.8276488101326512
67 0.8263032528774612
60 0.8301719460820514
54 0.8326190162679888
48 0.8287710682716157
43 0.8205559658176593


In [ ]:
#위의 코드로부터 최적의 피처를 얻어서..
model = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_1 = x_train[lgbm_archive_4040.iloc[4,2]]

model.fit(x_train_1, y_train)

pred_y1 = model.predict_proba(test[lgbm_archive_4040.iloc[4,2]])
pred_y1 = pred_y1[:,1]

In [ ]:
def lgbm_rfe_1234(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=1234)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive


In [ ]:
lgbm_archive_1234 = lgbm_rfe_1234(x_train, y_train)

426 0.8358568040047623
383 0.8376550282475487
344 0.8347018493065675
309 0.8351471676603491
278 0.8346782905291417
250 0.8297881204002925
225 0.8356140336764104
202 0.8356226527413222
181 0.8339979590054288
162 0.8327481945932032
145 0.8309539925807089
130 0.8282490427091904
117 0.8306764586905457
105 0.8290440077962316
94 0.8331366271185663
84 0.8317265480989788
75 0.8285690973195857
67 0.8205300954762551
60 0.8192237325377746
54 0.8134590146225309
48 0.818644244073531
43 0.821927533200638


In [ ]:
model2 = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_2 = x_train[lgbm_archive_1234.iloc[6,2]]

model2.fit(x_train_2, y_train)

pred_y2 = model2.predict_proba(test[lgbm_archive_1234.iloc[6,2]])
pred_y2 = pred_y2[:,1]

In [ ]:
def lgbm_rfe_99087(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=99087)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive


In [ ]:
lgbm_archive_99087 = lgbm_rfe_99087(x_train, y_train)

426 0.839307768912849
383 0.8436727369324074
344 0.8449808186349032
309 0.8387293380589103
278 0.8486897333698254
250 0.8422269472880448
225 0.8382929275040997
202 0.8400805433340156
181 0.8360444644282274
162 0.8370587308560087
145 0.8348608661053321
130 0.832808184049108
117 0.836528885893877
105 0.8315932492634495
94 0.8364567257823765
84 0.8353981858200493
75 0.8374281561280321
67 0.8328530325646221
60 0.8300203888251299
54 0.8266464005041433
48 0.8299082675363445
43 0.8313649818191018


In [ ]:
model3 = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_3 = x_train[lgbm_archive_99087.iloc[4,2]]

model3.fit(x_train_3, y_train)

pred_y3 = model3.predict_proba(test[lgbm_archive_99087.iloc[4,2]])
pred_y3 = pred_y3[:,1]

In [ ]:
def lgbm_rfe_42(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=42)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive


In [ ]:
lgbm_archive_42 = lgbm_rfe_42(x_train, y_train)

426 0.8490983808501918
383 0.8463304336613692
344 0.8469446376024057
309 0.8428829478105097
278 0.8507308854207442
250 0.8421553435167234
225 0.8467264714415316
202 0.8429730348927704
181 0.8410086759904398
162 0.8482625569304308
145 0.8408322434619228
130 0.8381676229904536
117 0.8464035394725329
105 0.8340207390249181
94 0.841476668309149
84 0.8393131392441204
75 0.8399978586329329
67 0.8407536691058296
60 0.8401210448029253
54 0.8396383737583525
48 0.8417368559205346
43 0.8409945729008846


In [ ]:
model4 = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_4 = x_train[lgbm_archive_42.iloc[4,2]]

model4.fit(x_train_4, y_train)

pred_y4 = model4.predict_proba(test[lgbm_archive_42.iloc[4,2]])
pred_y4 = pred_y4[:,1]

In [ ]:
pred_all = (pred_y1 + pred_y2 + pred_y3 + pred_y4) * (1/4)
submission = pd.DataFrame()
submission['index'] = submission.index
submission['nerdiness'] = pred_all
submission['index'] = submission.index
#submission = pd.DataFrame({
    #"index" : index,
 #   "nerdiness" : pred_all
#})

submission.to_csv('./drive/MyDrive/lee_test.csv', index=False)

## RandomForest

In [ ]:
def rf_rfe_4040(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = RandomForestClassifier()
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=4040)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [ ]:
rf_archive_4040 = rf_rfe_4040(x_train, y_train)

426 0.8584691304098533
383 0.8564977593734486
344 0.8541681013034618
309 0.8558529291513682
278 0.85674468497681
250 0.8562731636817097
225 0.8551811589713949
202 0.8548694045923552
181 0.8527882568291353
162 0.8557262969428767
145 0.8565474614348021
130 0.8570076016781721
117 0.8559446978849395
105 0.8564454641610681
94 0.8574160229649455
84 0.8565879434036144
75 0.8532074829118551
67 0.8498242852051227
60 0.8548796331325178
54 0.853058232663849
48 0.850842241047218
43 0.8518999585383964


In [ ]:
model1 = RandomForestClassifier()

x_train_1 = x_train[rf_archive_4040.iloc[14,2]]

model1.fit(x_train_1, y_train)

pred_y1 = model1.predict_proba(test[rf_archive_4040.iloc[14,2]])
pred_y1 = pred_y1[:,1]

In [ ]:
def rf_rfe_1234(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = RandomForestClassifier()
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=1234)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [ ]:
rf_archive_1234 = rf_rfe_1234(x_train, y_train)

426 0.8511509037376862
383 0.8525036659756092
344 0.8502526535227841
309 0.8520540380893716
278 0.8531236640449387
250 0.8532869953250191
225 0.8525905748801375
202 0.8512528960058104
181 0.8523666228435101
162 0.8507170774704539
145 0.8501383072616197
130 0.8492623229644067
117 0.8541967376264704
105 0.8508166276701864
94 0.8528729929070843
84 0.8535424069485753
75 0.8498716908536781
67 0.8515798458681353
60 0.8527474418615342
54 0.8516186316602388
48 0.8494572574824976
43 0.8506372074689368


In [ ]:
model2 = RandomForestClassifier()

x_train_2 = x_train[rf_archive_1234.iloc[12,2]]

model2.fit(x_train_2, y_train)

pred_y2 = model2.predict_proba(test[rf_archive_1234.iloc[12,2]])
pred_y2 = pred_y2[:,1]

In [ ]:
def rf_rfe_99087(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = RandomForestClassifier()
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=99087)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [ ]:
rf_archive_99087 = rf_rfe_99087(x_train, y_train)

426 0.8554726401056125
383 0.854656167130868
344 0.8543606269132492
309 0.8568842183823716
278 0.8585482133041397
250 0.8575914449731714
225 0.8574860797107615
202 0.8556531841296053
181 0.8554185918946082
162 0.8554354100879261
145 0.8571167981839801
130 0.8601577287791774
117 0.8604834554976231
105 0.8602699938132048
94 0.8575800890990507
84 0.8583836250020125
75 0.8583258394147154
67 0.8574185194470063
60 0.85660334017944
54 0.8571410911298836
48 0.854523490272472
43 0.8587478754453226


In [ ]:
model3 = RandomForestClassifier()

x_train_3 = x_train[rf_archive_99087.iloc[12,2]]

model3.fit(x_train_3, y_train)

pred_y3 = model3.predict_proba(test[rf_archive_99087.iloc[12,2]])
pred_y3 = pred_y3[:,1]

In [ ]:
def rf_rfe_42(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = RandomForestClassifier()
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=42)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [ ]:
rf_archive_42 = rf_rfe_42(x_train, y_train)

426 0.8666075104420427
383 0.8645979640895045
344 0.8658764523304061
309 0.8670795322149103
278 0.8655469005438613
250 0.8650292595935546
225 0.8665964294431063
202 0.864543422549286
181 0.8658269476078857
162 0.8662494646582332
145 0.8676902823380965
130 0.8671042845761705
117 0.8648125325234514
105 0.8667440801562047
94 0.8665997393518795
84 0.8687229738753217
75 0.8636674479970159
67 0.8649217595129657
60 0.8633504163001782
54 0.8628044252616843
48 0.8650589048634361
43 0.8656503711702916


In [ ]:
model4 = RandomForestClassifier()

x_train_4 = x_train[rf_archive_42.iloc[15,2]]

model4.fit(x_train_4, y_train)

pred_y4 = model4.predict_proba(test[rf_archive_42.iloc[15,2]])
pred_y4 = pred_y4[:,1]

In [ ]:
pred_all = (pred_y1 + pred_y2 + pred_y3 + pred_y4) * (1/4)
submission = pd.DataFrame()
submission['index'] = submission.index
submission['nerdiness'] = pred_all
submission['index'] = submission.index
#submission = pd.DataFrame({
    #"index" : index,
 #   "nerdiness" : pred_all
#})

submission.to_csv('./drive/MyDrive/lee_test.csv', index=False)

## Extra Trees

In [39]:
def ex_rfe_4040(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = ExtraTreesClassifier()
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=4040)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [52]:
ex_archive_4040 = ex_rfe_4040(x_train, y_train)

430 0.8626340190873204
387 0.8639363571587247
348 0.8624624389276917
313 0.8657025811360941
281 0.8630319237060392
252 0.8614862616299223
226 0.8656162868324689
203 0.8629430362513869
182 0.8639919658418621
163 0.864417098546366
146 0.8619390546120488
131 0.8621185582887048
117 0.8651946116626679
105 0.8657022930082021
94 0.86283830176268
84 0.8630762954013924
75 0.8629009695791691
67 0.8626497220574292
60 0.8654974340770587
54 0.8638605795231541
48 0.8619818416039964
43 0.864775529643894


In [41]:
model1 = ExtraTreesClassifier(n_estimators = 300)

x_train_1 = x_train[ex_archive_4040.iloc[9,2]]

model1.fit(x_train_1, y_train)

pred_y1 = model1.predict_proba(test[ex_archive_4040.iloc[9,2]])
pred_y1 = pred_y1[:,1]

In [42]:
def ex_rfe_1234(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = ExtraTreesClassifier()
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=1234)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [43]:
ex_archive_1234 = ex_rfe_1234(x_train, y_train)

430 0.85411198348817
387 0.8581281804349525
348 0.8581062018194271
313 0.8572586604364235
281 0.8546871624199576
252 0.8615450650681941
226 0.8572386929360443
203 0.8588501707724062
182 0.8573742995573249
163 0.8591463793032118
146 0.8571257831856983
131 0.8593391590550747
117 0.8618224553072753
105 0.8598094726971008
94 0.8584572850635053
84 0.8606631910766246
75 0.862108033658023
67 0.8611859373635314
60 0.8596528930178678
54 0.8623997890052909
48 0.8600529612808621
43 0.8607676254131406


In [44]:
model2 = ExtraTreesClassifier(n_estimators = 300)

x_train_2 = x_train[ex_archive_1234.iloc[8,2]]

model2.fit(x_train_2, y_train)

pred_y2 = model2.predict_proba(test[ex_archive_1234.iloc[8,2]])
pred_y2 = pred_y2[:,1]

In [45]:
def ex_rfe_99087(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = ExtraTreesClassifier()
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=99087)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [46]:
ex_archive_99087 = ex_rfe_99087(x_train, y_train)

430 0.86256747401661
387 0.8629297120265319
348 0.865448272412183
313 0.8629320119504045
281 0.8615675821130321
252 0.8617600569921137
226 0.8640021952773362
203 0.863337373532936
182 0.864036262899698
163 0.8667746097604169
146 0.8654235482305536
131 0.8656072546498711
117 0.8633976027893477
105 0.8654954208515697
94 0.8655559375984655
84 0.8666417891567789
75 0.8677027727882207
67 0.8662955068687226
60 0.8648265742403927
54 0.8671634406401147
48 0.8626744204766822
43 0.8660251220684596


In [47]:
model3 = ExtraTreesClassifier(n_estimators = 300)

x_train_3 = x_train[ex_archive_99087.iloc[11,2]]

model3.fit(x_train_3, y_train)

pred_y3 = model3.predict_proba(test[ex_archive_99087.iloc[11,2]])
pred_y3 = pred_y3[:,1]

In [48]:
def ex_rfe_42(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = ExtraTreesClassifier()
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=42)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [49]:
ex_archive_42 = ex_rfe_42(x_train, y_train)

430 0.8727832245764469
387 0.8739071544685496
348 0.8718151482148366
313 0.870516368794065
281 0.876549181214915
252 0.8705379551556289
226 0.873274242347491
203 0.8748146451087031
182 0.8720297166487835
163 0.8751873696183762
146 0.8769679566292581
131 0.8757054422959138
117 0.8753305591500846
105 0.8753586214201178
94 0.8732956847999779
84 0.8754956228615112
75 0.87560758412349
67 0.8769862330820489
60 0.8759423166368103
54 0.8747796752029694
48 0.8764461423157158
43 0.8737494301200548


In [50]:
model4 = ExtraTreesClassifier(n_estimators = 300)

x_train_4 = x_train[ex_archive_42.iloc[11,2]]

model4.fit(x_train_4, y_train)

pred_y4 = model4.predict_proba(test[ex_archive_42.iloc[11,2]])
pred_y4 = pred_y4[:,1]

In [51]:
pred_all = (pred_y1 + pred_y2 + pred_y3 + pred_y4) * (1/4)
submission = pd.DataFrame()
submission['index'] = submission.index
submission['nerdiness'] = pred_all
submission['index'] = submission.index
#submission = pd.DataFrame({
    #"index" : index,
 #   "nerdiness" : pred_all
#})

submission.to_csv('./drive/MyDrive/lee_test.csv', index=False)